#Amazon Review Analysis

<b> Objective: </b> : To determine whether a review is positive or negative and build based on the rating provided. <br>
<b> Data Source :</b> https://www.kaggle.com/snap/amazon-fine-food-reviews/

<br> <b>Data Overview :</b> This dataset contains all the reviews written by people who purchaged iffrent food product from the amazon along with their rating and review<br>
<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 <br>




Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review




<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is  ignored as it can be positive or negative and that can impact our model accuracy. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




##Loding Dataset
The dataset is in 2 format 
1. Sqlite Database
2. .csv file

We will be using Sqlite database as it is more easier to fetch,visulize and do manipulation of data with sql commands


In [0]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os


In [0]:
con=sqlite3.connect('Amazondatabase.sqlite')

**Reading database into dataframe by neglecting all the 3 star reviews**

In [0]:
data=pd.read_sql_query("""
SELECT * 
From reviews
where Score !=3
""",con)

In [4]:
#Lets See how Our Dataframe Looks Like
data.head(4)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...


#Converting Score into positive or negative
We are going to change our score in the form of positive and negative<br>
Score 1-2 : Negative<br>
Score 4-5 : Positive<br>
Score 3 : Already Neglected


In [0]:
## Writting function to convert score into Positive or Negative
## Positive -> 1 and Negative -> 0
def convert(x):
  if x >3:
    return 1
  return 0

In [0]:
score=data.Score
new=score.map(convert)

In [0]:
#new column after geting score converted in Positive or Negative
# Updating our score  column with  0 & 1 \\ the new colum that we have ceated
data.Score=new

In [8]:
data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


*Score are coverted into Positve and Negative*

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525814 entries, 0 to 525813
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      525814 non-null  int64 
 1   ProductId               525814 non-null  object
 2   UserId                  525814 non-null  object
 3   ProfileName             525814 non-null  object
 4   HelpfulnessNumerator    525814 non-null  int64 
 5   HelpfulnessDenominator  525814 non-null  int64 
 6   Score                   525814 non-null  int64 
 7   Time                    525814 non-null  int64 
 8   Summary                 525814 non-null  object
 9   Text                    525814 non-null  object
dtypes: int64(5), object(5)
memory usage: 40.1+ MB


##[2] ***Exploratory Data Analysis***
#Data Cleaning *(Deduplication)*
After having  look into data we discovered that there were many duplicacy in data like <br>
Same Person reviewed many product with same time stamp and given similar review and rating. 
<br>
So it is very important to remove those type of duplicate data in order to prevent unbiased results for the analysis of the data. 

*Lets have look at some duplications*

In [10]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


*Here we can see that same person have reviewdor rated many diffrent product(with diffrent product id at the same timestamp with same review and same rating which is imposible as no once can give review to diffrent products at the same time*<br>

**So we need to remove this Duplication to get more acuurate result**

In [0]:
sort_data=data.sort_values(by=['ProductId'],axis=0,ascending =True)
clean=data.drop_duplicates(subset={'ProductId','UserId','ProfileName','Score','Time','Summary'},keep='first',inplace=False)

In [12]:
print("Before Cleaning: ",data.shape)
print("After Cleaning: ",clean.shape)
print("Data Lost :",data.shape[0]-clean.shape[0])

Before Cleaning:  (525814, 10)
After Cleaning:  (524507, 10)
Data Lost : 1307


**As we know that the HelpfulnessNumerator is the number of users who found the review helpful and HelpfulnessDenominator is number of users who indicated whether they found the review helpful or not** <br>
*So here we can conclude that HelpfulnessDenominator should always be greater than the HelpfulnessNumerator*<br>

Lets see if there is any data that is voilating up this rule

In [13]:
display=pd.read_sql_query('''
Select * 
from Reviews
where Score !=3 and HelpfulnessDenominator < HelpfulnessNumerator ''',con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
1,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...




We found two row that is not matching this condition so we need to remove them to



In [0]:
final=clean[clean.HelpfulnessDenominator>=clean.HelpfulnessNumerator]

Now we have perform data cleaning and our cleaned dataframe is saved in **final** datafram

In [0]:
text=final.Text

In [16]:
print(text)

0         I have bought several of the Vitality canned d...
1         Product arrived labeled as Jumbo Salted Peanut...
2         This is a confection that has been around a fe...
3         If you are looking for the secret ingredient i...
4         Great taffy at a great price.  There was a wid...
                                ...                        
525809    Great for sesame chicken..this is a good if no...
525810    I'm disappointed with the flavor. The chocolat...
525811    These stars are small, so you can give 10-15 o...
525812    These are the BEST treats for training and rew...
525813    I am very satisfied ,product is as advertised,...
Name: Text, Length: 524505, dtype: object


In [17]:
#for the very last lets check is there any  null value present 
final.isna().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

## [3]Text pre-processing
<br> Preprocessing Review Text
<br> Now when we have finally removed all the duplicates and all unwanted data now we need to do some pre-processing of our data before creating our model

In [18]:
text=final.Text
text.head()

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
Name: Text, dtype: object

<b> Lets first check out what type of garbage are there in our text </b> <br>
We re going to see the text of random text columm to have a look at our text data garbage



In [19]:
import random
i=0
while i <5:
    
    ran=random.randint(1,1000)
    print("Found at: ",ran)
    print(text.values[ran])
    print('-'*100)
    i+=1

Found at:  156
I have drunk Ricore since my mother allowed be to drink caffeine. I love this stuff in milk.<br />It's much smoother and tastier than instant coffee with milk. I am French and live in the US,<br />and bring back boxes of Ricore at every opportunity. Savora is another hidden gem you can bring<br />back. Check it out.
----------------------------------------------------------------------------------------------------
Found at:  724
This is NOT Disco Dust which is edible and made for baking. DO NOT purchase this product for such purposes!
----------------------------------------------------------------------------------------------------
Found at:  529
I have never been addicted to anything in my life...until I tasted these chips.  I have tried other brands of the sea salt and vinegar flavor and they are just not the same.  You've got to stick with this blue bag of chips!
----------------------------------------------------------------------------------------------------
Fo

Here we can see that some of our text have **HTML** tag  within it<br>
So lets remove all the html tags using simple regualr expression 

There might also be url as people somtimes refer another product.<br>
So lets remove all url

In [0]:
## function to remove URl
def removeUrl(text):
  pr=re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  return pr
## we also use Beautifull Soup to do this more efficiently

Now we need to de concatinate words like:<br>
won't -> will no<br>
there're -> there are

In [0]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [22]:
# Without using decontracted function
i=0
n=[]
while i<5:
  n.append(random.randint(100,1000))
  print(text.values[n[i]])
  i+=1

It feels strange to review chips, but I am compelled because these are my absolute favorite chips. Kettle has lots of flavors but I always go back to these regular lightly salted.<br /><br />They are thicker and sometimes greasy at the bottom of the bag, plus they often looked burned. They are not actually burned. It has something to do with how much sugar is in the potato that makes it dark. I love to open a bag and find some browned chips.<br /><br />Lays and Wise have their place but I've become a chip snob thanks to Kettle.<br /><br />Like Edy's or Breyer's ice cream, some products are worth the money. These are the best of the best.
It took a while to get used to these chips but they are pretty good and more healthy for you. They sure were eaten all up by my family and friends.
If you want to find a good flavor substitute for maple syrup, barley malt, or brown rice syrup, this will do nicely. I use Madhava's agave nectar in my baked goods all the time, and am always pleased with t

Here we also see  numbers so we need to remove them too

In [23]:
#after using  all function
from bs4 import BeautifulSoup
for i in n:
  cleanHtml= BeautifulSoup(text.values[i], 'lxml').get_text() ## Remove Html Tags
  cleanUrl=removeUrl(cleanHtml) #remove
  cleanNums = re.sub(r'[0-9]+', '', cleanUrl)   ## To Remove Numbers from the string
  low=cleanNums.lower()
  cleanChar = re.sub(r'[^a-z0-9\s]', '', low)   ## To clean all special Charaters
  AddCleaned=decontracted(cleanChar)
  print(AddCleaned)

it feels strange to review chips but i am compelled because these are my absolute favorite chips kettle has lots of flavors but i always go back to these regular lightly saltedthey are thicker and sometimes greasy at the bottom of the bag plus they often looked burned they are not actually burned it has something to do with how much sugar is in the potato that makes it dark i love to open a bag and find some browned chipslays and wise have their place but ive become a chip snob thanks to kettlelike edys or breyers ice cream some products are worth the money these are the best of the best
it took a while to get used to these chips but they are pretty good and more healthy for you they sure were eaten all up by my family and friends
if you want to find a good flavor substitute for maple syrup barley malt or brown rice syrup this will do nicely i use madhavas agave nectar in my baked goods all the time and am always pleased with the results at the time im writing this amazon was out of st

<b>Stemming -></b> Its the process  of reducing inflected words to their word stem, base or root form—generally a written word form

In [0]:
from nltk.stem import PorterStemmer 

In [0]:
snow=nltk.stem.SnowballStemmer('english')

In [26]:
snow.stem('hacked')

'hack'

##Stop Words **Removal**
<br>
A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore, both when indexing entries for searching and when retrieving them as the result of a search query.<br>
<b>Why to remove stopwords?</b>
<br>
Removing stopwords can potentially help improve the performance as there are fewer and only meaningful tokens left. 

In [0]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [28]:
print("Text With Stop Words")
sample=final.Text.values[1000]
print("-"*80)
print(sample)

Text With Stop Words
--------------------------------------------------------------------------------
All my fault. I thought this would be a carton of 3 boxes of cereal.  Instead it was 1 box of cereal for 12.99. Who sells cereal for this price plus $8.00 shipping ? I feel stupid!  Buyer Beware!


Lets see how to remove them

In [29]:
print("After Removing StopWords")
print("-"*80)
token=sample.split(" ")   ## coverting string to token (list of word) \\ like ["this","is","token"]
removestop=[x for x in token if x not in stopwords]   ##removing all the words from the token that are stopwords
removed=" ".join(removestop)  ##joing back the list into sentence
print(removed)

After Removing StopWords
--------------------------------------------------------------------------------
All fault. I thought would carton 3 boxes cereal.  Instead 1 box cereal 12.99. Who sells cereal price plus $8.00 shipping ? I feel stupid!  Buyer Beware!


Now lets just write a small function to remove <b>Stopword and do Stemming

In [0]:
def removeStopWord(word):
  token=word.split(" ")   ## coverting string to token (list of word) \\ like ["this","is","token"]
  removestop=[snow.stem(x) for x in token if x not in stopwords]   ##removing stopwords and also doing Stemming
  removed=" ".join(removestop)  ##joing back the list into sentence
  return removed

**Now lets just create a list containg all cleaned text after applying all the filter that we have discussed**

In [31]:
from tqdm import tqdm
preprocessed_reviews = []
for line in tqdm(final.Text.values):
  line= BeautifulSoup(line, 'lxml').get_text() ## Remove Html Tags
  line=removeUrl(line) #removing url
  line=decontracted(line)    #Coverting word like { are't -> are not}
  line = re.sub(r'[0-9]+', '', line)   ## To Remove Numbers from the string
  line=line.lower()   ## Converting every word to lower case
  line = re.sub(r'[^a-z0-9\s]', '', line)   ## To clean all special Charaters
  line=removeStopWord(line)    ## Removing Stop Words And doing Steaming
  preprocessed_reviews.append(line.strip()) ## ading cleaned word into a list after removing spaces {By using strip()}

100%|██████████| 524505/524505 [07:31<00:00, 1162.77it/s]


In [32]:
#just to check evry thing went correctly after pre-processing
print("preprocessed list size = ",len(preprocessed_reviews))
print("Normal list size = ",final.shape[0])

preprocessed list size =  524505
Normal list size =  524505


# [4]Featurization<br>
<b>Bag Of Words

Lets just see with a toy example how theBag Of Words actually looks like

In [33]:
## uni-gram
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)


['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
(4, 9)


In [34]:
## by-gram
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)


['and', 'and this', 'document', 'document is', 'first', 'first document', 'is', 'is the', 'is this', 'one', 'second', 'second document', 'the', 'the first', 'the second', 'the third', 'third', 'third one', 'this', 'this document', 'this is', 'this the']
(4, 22)


## Modeling

In [0]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection  import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection  import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

Spliting data it into train and test

In [0]:
X=preprocessed_reviews

y=np.array(final['Score'])
X_1, X_test, y_1, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train, X_cv, y_train, y_cv = train_test_split(X_1, y_1, test_size=0.3)

In [56]:
X_train[1]

'tri waffl mix bought new waffl maker amaz tasti waffl would go bother measur ingredi small number waffl use mix besid conveni waffl absolut delici not imagin not mix hand pantri definit winner'

## Using Bag of Word 

In [0]:
count_vect = CountVectorizer()
X_train=count_vect.fit_transform(X_train)
X_cv=count_vect.transform(X_cv)
X_test=count_vect.transform(X_test)



### Using Standard Scaler to  Standardize features by removing the mean and scaling to unit variance

In [0]:
scalar = StandardScaler(with_mean=False)
X_train = scalar.fit_transform(X_train)
X_test= scalar.transform(X_test)
X_cv=scalar.transform(X_cv)

# Model Building
## Logestic Regression

In [0]:
c = 10**-4
lr=LogisticRegression(penalty='l2',C=c)

In [65]:
lr.fit(X_train,y_train)

LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
probcv=lr.predict_proba(X_cv)[:,1]

In [70]:
predictions = lr.predict(X_cv)
print('AUC: ', roc_auc_score(y_cv, predictions))

AUC:  0.7818346168856577


## Mode have Accurracy of 78 % lets try to increses this

DOing Same Opereartion Using Tf-Idf

In [0]:
X=preprocessed_reviews

y=np.array(final['Score'])
X_1, X_test, y_1, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train, X_cv, y_train, y_cv = train_test_split(X_1, y_1, test_size=0.3)

In [110]:
print(X_test[5])
print(y_test[5])

buy dogswel chew treat dog sever year alway love recent time purchas howev got serious ill heavi vomit lethargi almost forgot week later found back cabinet tri got ill especi sinc rare sick difficult ignor link treat ill  unless qualiti control high recommend find differ treat issu  amad china look packag care read review factth packag clear state product made china find pretti incredul consid claim packag natur even free rang defin bewar
0


### Using Bi-Grams

In [0]:
count_vect = CountVectorizer(min_df = 5, ngram_range = (1,2))
X_train=count_vect.fit_transform(X_train)
X_cv=count_vect.transform(X_cv)
X_test=count_vect.transform(X_test)


In [0]:
scalar = StandardScaler(with_mean=False)
X_train = scalar.fit_transform(X_train)
X_test= scalar.transform(X_test)
X_cv=scalar.transform(X_cv)

In [106]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [108]:
predictions = lr.predict(X_test)
print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.8851395124283175


### Now we get pretty good acuraccy of 88%

#### Lets just check it mannuly

In [0]:
text=['I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most','Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".']
text=count_vect.transform(text)
text=scalar.transform(text)

In [121]:
print(lr.predict(text))

[1 0]
